In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_excel('/content/updated_duplicated_spam_dataset (1).xlsx')

# Inspect the first few rows
print(df.head())

# Check for missing values
print(df.isnull().sum())


    v1                                                 v2 language     source
0  ham          जोस पूछते हैं कि क्या आप मिलना चाहते हैं?    Hindi     Google
1  ham  ఆయన అక్కడే ఉన్నారు. మీరు అతడికి ఫోన్ చేసి కలుస...   Telugu  Microsoft
2  ham  ఉపాధ్యాయుడి కంటే జీవితం చాలా కఠినమైనది ... Bco...   Telugu     Google
3  ham  बहुत खूब। आप ठीक कह रहे हैं! मुझे ऐसा करने का ...    Hindi     Google
4  ham               आप अपनी बाइक की सवारी कब कर रहे हैं?    Hindi     Google
v1          0
v2          0
language    0
source      0
dtype: int64


In [ ]:
pip install nltk indic-nlp-library


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from indicnlp.tokenize import indic_tokenize

# Download necessary NLTK resources
nltk.download('punkt')

# Function to clean text (preserving Hindi and Telugu scripts)
def clean_text(text):
    # Remove special characters, numbers, and extra spaces but preserve scripts
    # This allows Hindi/Telugu characters to remain intact
    text = re.sub(r'[^a-zA-Z\s\u0900-\u097F\u0C00-\u0C7F]', '', text, re.UNICODE)
    text = text.strip()  # Remove leading/trailing spaces
    return text

# Function to tokenize text (supports English, Hindi, Telugu)
def tokenize_text(text, language):
    if language == 'English':
        return word_tokenize(text)  # Tokenize in English
    else:
        return indic_tokenize.trivial_tokenize(text)  # Tokenize Hindi/Telugu using Indic NLP

# Apply the cleaning function to the 'v2' column (SMS text)
df['cleaned_text'] = df['v2'].apply(clean_text)

# Tokenize the 'cleaned_text' column based on language (either Hindi or Telugu)
df['tokens'] = df.apply(lambda row: tokenize_text(row['cleaned_text'], row['Language']), axis=1)

# Inspect the cleaned and tokenized data
print(df[['v2', 'cleaned_text', 'tokens']].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


KeyError: 'Language'

In [ ]:
print(df.columns)


Index(['v1', 'v2', 'language', 'source', 'cleaned_text'], dtype='object')


In [ ]:
# Modify the tokenization function to refer to the 'language' column correctly
df['tokens'] = df.apply(lambda row: tokenize_text(row['cleaned_text'], row['language']), axis=1)

# Inspect the cleaned and tokenized data
print(df[['v2', 'cleaned_text', 'tokens']].head())


                                                  v2  \
0          जोस पूछते हैं कि क्या आप मिलना चाहते हैं?   
1  ఆయన అక్కడే ఉన్నారు. మీరు అతడికి ఫోన్ చేసి కలుస...   
2  ఉపాధ్యాయుడి కంటే జీవితం చాలా కఠినమైనది ... Bco...   
3  बहुत खूब। आप ठीक कह रहे हैं! मुझे ऐसा करने का ...   
4               आप अपनी बाइक की सवारी कब कर रहे हैं?   

                                        cleaned_text  \
0           जोस पूछते हैं कि क्या आप मिलना चाहते हैं   
1  ఆయన అక్కడే ఉన్నారు మీరు అతడికి ఫోన్ చేసి కలుస్...   
2  ఉపాధ్యాయుడి కంటే జీవితం చాలా కఠినమైనది  Bcoz ఉ...   
3  बहुत खूब। आप ठीक कह रहे हैं मुझे ऐसा करने का म...   
4                आप अपनी बाइक की सवारी कब कर रहे हैं   

                                              tokens  
0  [जोस, पूछते, हैं, कि, क्या, आप, मिलना, चाहते, ...  
1  [ఆయన, అక్కడే, ఉన్నారు, మీరు, అతడికి, ఫోన్, చేస...  
2  [ఉపాధ్యాయుడి, కంటే, జీవితం, చాలా, కఠినమైనది, B...  
3  [बहुत, खूब, ।, आप, ठीक, कह, रहे, हैं, मुझे, ऐस...  
4      [आप, अपनी, बाइक, की, सवारी, कब, कर, रहे, हैं

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Join the tokens back into a string (as TF-IDF works on text, not tokens)
df['text_for_vectorization'] = df['tokens'].apply(lambda x: ' '.join(x))

# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can tune the number of features

# Fit and transform the training data (we will split later)
X_tfidf = tfidf_vectorizer.fit_transform(df['text_for_vectorization'])

# Save the TF-IDF Vectorizer for future use (so that you can transform new text data)
import pickle
with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

print("TF-IDF Vectorization applied successfully!")


TF-IDF Vectorization applied successfully!


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and labels (y)
X = X_tfidf
y = df['v1'].apply(lambda x: 1 if x == 'spam' else 0)  # 1 for spam, 0 for ham

# Split the data into train, validation, and test sets (80%, 10%, 10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Save the split data into a dictionary and pickle it
train_val_test_data = {
    'X_train': X_train, 'y_train': y_train,
    'X_val': X_val, 'y_val': y_val,
    'X_test': X_test, 'y_test': y_test
}

with open("train_val_test_data.pkl", "wb") as f:
    pickle.dump(train_val_test_data, f)

print("Data split into training, validation, and test sets.")


Data split into training, validation, and test sets.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize Logistic Regression model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Make predictions on validation set
y_val_pred = model.predict(X_val)

# Evaluate the model
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))

# Save the trained model for future use
with open("text_classification_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model trained and saved successfully!")


Validation Accuracy: 0.9137055837563451
              precision    recall  f1-score   support

           0       0.90      0.99      0.94       143
           1       0.95      0.72      0.82        54

    accuracy                           0.91       197
   macro avg       0.93      0.85      0.88       197
weighted avg       0.92      0.91      0.91       197

Model trained and saved successfully!


In [ ]:
# Make predictions on the test set
y_test_pred = model.predict(X_test)

# Evaluate on test set
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


Test Accuracy: 0.9390862944162437
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       135
           1       1.00      0.81      0.89        62

    accuracy                           0.94       197
   macro avg       0.96      0.90      0.93       197
weighted avg       0.94      0.94      0.94       197



In [ ]:
from sklearn.model_selection import GridSearchCV

# Hyperparameter grid for tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}

# Grid Search with cross-validation
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=3)
grid_search.fit(X_train, y_train)

# Best parameters from grid search
print("Best Hyperparameters:", grid_search.best_params_)

# Use the best model
best_model = grid_search.best_estimator_

# Save the best model
with open("best_text_classification_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Hyperparameter tuning complete and best model saved!")


Best Hyperparameters: {'C': 10, 'solver': 'saga'}
Hyperparameter tuning complete and best model saved!


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# Initialize StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(LogisticRegression(max_iter=1000), X, y, cv=cv, scoring='accuracy')

print("Cross-validation scores:", cv_scores)
print("Mean cross-validation accuracy:", cv_scores.mean())


Cross-validation scores: [0.9213198  0.92639594 0.93147208 0.93401015 0.91603053]
Mean cross-validation accuracy: 0.9258457007788584


In [ ]:
# Re-evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)

# Evaluate on the test set
from sklearn.metrics import accuracy_score, classification_report
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))


Test Accuracy: 0.9847715736040609
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       135
           1       0.98      0.97      0.98        62

    accuracy                           0.98       197
   macro avg       0.98      0.98      0.98       197
weighted avg       0.98      0.98      0.98       197



In [ ]:
# Save the final model
with open("final_text_classification_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Final model saved successfully!")


Final model saved successfully!


In [ ]:
import pickle
import re
from indicnlp.tokenize import indic_tokenize

# Load the trained model and TF-IDF vectorizer
with open('/content/final_text_classification_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

with open('tfidf_vectorizer.pkl', 'rb') as vectorizer_file:
    tfidf_vectorizer = pickle.load(vectorizer_file)

# Preprocessing function for new input text
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = text.strip()  # Trim spaces
    tokens = indic_tokenize.trivial_tokenize(text)  # Tokenize Hindi and Telugu text
    return " ".join(tokens)

# Input: Manually enter Hindi or Telugu text for testing
input_texts = [
    "क्या आप इस शुक्रवार को मुझसे मिलना चाहेंगे?",  # Hindi
    "మీరు నాకు కాల్ చేసి మాట్లాడవచ్చా?"  # Telugu
]

# Preprocess input texts
processed_texts = [preprocess_text(text) for text in input_texts]

# Transform using the TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(processed_texts)

# Predict using the trained model
predictions = model.predict(X_test_tfidf)

# Display results
for text, pred in zip(input_texts, predictions):
    label = "Spam" if pred == 1 else "Ham"
    print(f"Text: {text}\nPredicted Label: {label}\n")


Text: क्या आप इस शुक्रवार को मुझसे मिलना चाहेंगे?
Predicted Label: Ham

Text: మీరు నాకు కాల్ చేసి మాట్లాడవచ్చా?
Predicted Label: Ham



# **SVM**

In [ ]:
pip install transformers torch sentencepiece scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 802.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load MuRIL model
MODEL_NAME = "google/muril-base-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

In [ ]:
def get_muril_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # CLS token representation


In [ ]:
import pandas as pd
import numpy as np

# Load your dataset
df = pd.read_excel("/content/updated_duplicated_spam_dataset (1).xlsx")  # Update with actual dataset file

# Convert each text message to embeddings
df["embeddings"] = df["v2"].apply(get_muril_embedding)

# Convert list of embeddings to a NumPy array
X = np.vstack(df["embeddings"].values)
y = df["v1"].map({"ham": 0, "spam": 1})  # Convert labels to 0 (ham) and 1 (spam)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM classifier
svm_clf = SVC(kernel="linear", C=1.0, probability=True, random_state=42)
svm_clf.fit(X_train, y_train)

# Evaluate
y_pred = svm_clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.71      1.00      0.83       278
           1       0.00      0.00      0.00       116

    accuracy                           0.71       394
   macro avg       0.35      0.50      0.41       394
weighted avg       0.50      0.71      0.58       394



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
svm_clf = SVC(kernel="linear", C=1.0, probability=True, class_weight="balanced", random_state=42)
svm_clf.fit(X_train, y_train)


SVC(class_weight='balanced', kernel='linear', probability=True, random_state=42)

In [ ]:
svm_clf = SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, class_weight="balanced", random_state=42)


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "C": [0.1, 1, 10],
    "kernel": ["linear", "rbf"],
    "gamma": ["scale", "auto"]
}

svm_grid = GridSearchCV(SVC(probability=True, class_weight="balanced"), param_grid, cv=3)
svm_grid.fit(X_train, y_train)

print("Best parameters:", svm_grid.best_params_)


Best parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}


In [ ]:
# Train the SVM model using the best parameters
svm_clf_optimized = SVC(kernel="rbf", C=10, gamma="scale", probability=True, class_weight="balanced", random_state=42)
svm_clf_optimized.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_clf_optimized.predict(X_test)

# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9695431472081218

Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.98       278
           1       0.94      0.96      0.95       116

    accuracy                           0.97       394
   macro avg       0.96      0.97      0.96       394
weighted avg       0.97      0.97      0.97       394



In [ ]:
import pickle

# Save the trained SVM model to a file
with open('svm_model_optimized.pkl', 'wb') as f:
    pickle.dump(svm_clf_optimized, f)

print("✅ SVM Model saved successfully!")


✅ SVM Model saved successfully!


In [ ]:
import pickle

# Load the saved SVM model
with open('svm_model_optimized.pkl', 'rb') as f:
    svm_model = pickle.load(f)

# Load the TF-IDF vectorizer used during training
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)


In [ ]:
import nltk

# Ensure the correct punkt resource is downloaded
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re

# Function for text preprocessing using regex tokenization
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove non-alphabetic characters, keeping spaces and words
    text = re.sub(r'[^a-zA-Z\s\u0900-\u097F\u0C00-\u0C7F]', '', text)  # For Hindi and Telugu characters

    # Tokenization using regex
    tokens = re.findall(r'\b\w+\b', text)

    # Return the preprocessed text
    return ' '.join(tokens)

# Test the function with both Hindi and Telugu examples

# Example input text for Hindi
input_text_hindi = "जोस पूछते हैं कि क्या आप मिलना चाहते हैं?"

# Example input text for Telugu
input_text_telugu = "ఆయన అక్కడే ఉన్నారు. మీరు అతడికి ఫోన్ చేసి కలుసుకోవచ్చు."

# Preprocess the input text
processed_text_hindi = preprocess_text(input_text_hindi)
processed_text_telugu = preprocess_text(input_text_telugu)

# Print the processed text
print(f"Processed Hindi Text: {processed_text_hindi}")
print(f"Processed Telugu Text: {processed_text_telugu}")


Processed Hindi Text: ज स प छत ह क क य आप म लन च हत ह
Processed Telugu Text: ఆయన అక కడ ఉన న ర మ ర అతడ క ఫ న చ స కల స క వచ చ


In [ ]:
import joblib

# Load the trained SVM model
svm_model = joblib.load('/content/svm_model_optimized.pkl')

# Load the saved TF-IDF vectorizer
vectorizer = joblib.load('/content/tfidf_vectorizer.pkl')

# Input text for testing
input_text = "Your sample input message here."

# Preprocess the input text (same preprocessing used during training)
processed_text = preprocess_text(input_text)

# Vectorize the preprocessed text using the same vectorizer
vectorized_text = vectorizer.transform([processed_text])

# Check the shape of the vectorized text to ensure it's consistent
print(f"Shape of vectorized text: {vectorized_text.shape}")

# Get the prediction from the model
prediction = svm_model.predict(vectorized_text)

# Print the result
if prediction == 0:
    print("This is Ham.")
else:
    print("This is Spam.")


Shape of vectorized text: (1, 1694)


ValueError: X has 1694 features, but SVC is expecting 768 features as input.

In [ ]:
# Check the number of features in the vectorizer
print(f"Number of features in the vectorizer: {vectorizer.get_feature_names_out().shape[0]}")


Number of features in the vectorizer: 1694


# ** Retrain the same SVM Model**

In [ ]:
import pandas as pd
import pickle
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Step 1: Load the original dataset
data = pd.read_excel('/content/updated_duplicated_spam_dataset (1).xlsx')

# Step 2: Preprocess the text
def preprocess_text(text):
    # Your text preprocessing steps here (lowercase, remove stopwords, punctuation, etc.)
    return text

data['processed_text'] = data['v2'].apply(preprocess_text)

# Step 3: Vectorize the text using the original vectorizer
with open('/content/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

X = tfidf_vectorizer.transform(data['processed_text'])
y = data['v1']  # Assuming 'v1' is the target variable (ham/spam)

# Step 4: Train the SVM model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_model = SVC(C=10, gamma='scale', kernel='rbf')  # Using the best parameters found earlier
svm_model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

# Save the retrained SVM model
with open('/content/svm_model.pkl', 'wb') as f:
    pickle.dump(svm_model, f)


              precision    recall  f1-score   support

         ham       1.00      1.00      1.00       278
        spam       0.99      0.99      0.99       116

    accuracy                           0.99       394
   macro avg       0.99      0.99      0.99       394
weighted avg       0.99      0.99      0.99       394



In [ ]:
import pickle

# Step 1: Load the retrained SVM model
with open('/content/svm_model.pkl', 'rb') as f:
    svm_model = pickle.load(f)

# Load the vectorizer
with open('/content/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

# Function to preprocess and predict using the SVM model
def predict_spam(input_text):
    # Preprocess the text (apply the same preprocessing steps used during training)
    processed_text = preprocess_text(input_text)

    # Vectorize the input text
    vectorized_text = tfidf_vectorizer.transform([processed_text])

    # Predict using the SVM model
    prediction = svm_model.predict(vectorized_text)

    return "Spam" if prediction[0] == 1 else "Ham"

# Test the model with a sample input
input_text = "ఇదిగో మీ డిస్కౌంట్ కోడ్ RP176781. తదుపరి సందేశాలను ఆపడానికి రిప్లై ఆపండి. www.regalportfolio.co.uk. కస్టమర్ సర్వీసెస్ 08717205546"
result = predict_spam(input_text)
print(f"The message is: {result}")


The message is: Ham


In [ ]:
input_texts = [
    "आपको इस एसएमएस को ध्यान से पढ़ने की आवश्यकता है",  # Example in Hindi
    "మీరు ఈ సందేశాన్ని మరింతగా చదవాలి",  # Example in Telugu
    "Get a free gift card today! Visit our website.",  # Example with English
]

for text in input_texts:
    result = predict_spam(text)
    print(f"The message '{text}' is classified as: {result}")


The message 'आपको इस एसएमएस को ध्यान से पढ़ने की आवश्यकता है' is classified as: Ham
The message 'మీరు ఈ సందేశాన్ని మరింతగా చదవాలి' is classified as: Ham
The message 'Get a free gift card today! Visit our website.' is classified as: Ham


In [ ]:
from sklearn.metrics import classification_report

# Assuming you have a test set (X_test, y_test)
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         ham       1.00      1.00      1.00       278
        spam       0.99      0.99      0.99       116

    accuracy                           0.99       394
   macro avg       0.99      0.99      0.99       394
weighted avg       0.99      0.99      0.99       394



# **The Random Forest**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import pickle
import nltk
nltk.download('punkt')

# Load the CSV dataset
df = pd.read_excel('/content/updated_duplicated_spam_dataset (1).xlsx')  # replace with your actual path

# Check the structure of the dataset
print(df.head())

# Preprocess the text data (similar to how we did for SVM)
def preprocess_text(text):
    # Tokenize and remove unnecessary characters or stop words
    tokens = nltk.word_tokenize(text.lower())  # Tokenizing and converting to lowercase
    return " ".join(tokens)

# Preprocess the text column
df['processed_text'] = df['text_column'].apply(preprocess_text)  # Replace 'text_column' with your column name

# Define the features (X) and labels (y)
X = df['processed_text']
y = df['label_column']  # Replace 'label_column' with the column name containing labels (Spam/Ham)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=1000)  # Limiting to top 1000 features
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Save the vectorizer for later use
with open('/content/rf_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Check the shape of the vectorized data
print(X_train_vec.shape)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


    v1                                                 v2 language     source
0  ham          जोस पूछते हैं कि क्या आप मिलना चाहते हैं?    Hindi     Google
1  ham  ఆయన అక్కడే ఉన్నారు. మీరు అతడికి ఫోన్ చేసి కలుస...   Telugu  Microsoft
2  ham  ఉపాధ్యాయుడి కంటే జీవితం చాలా కఠినమైనది ... Bco...   Telugu     Google
3  ham  बहुत खूब। आप ठीक कह रहे हैं! मुझे ऐसा करने का ...    Hindi     Google
4  ham               आप अपनी बाइक की सवारी कब कर रहे हैं?    Hindi     Google


KeyError: 'text_column'

In [ ]:
# Check the column names of the dataset
print(df.columns)


Index(['v1', 'v2', 'language', 'source'], dtype='object')


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Manually download punkt tokenizer
import nltk
nltk.download('punkt')

# Preprocess the text column
df['processed_text'] = df['v2'].apply(preprocess_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
import nltk

# Try to download the punkt tokenizer again (just in case there's an issue)
nltk.download('punkt')

# Now retry running your previous code
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK resources
nltk.download('stopwords')

# Load your Excel file (adjust the path as needed)
file_path = '/content/updated_duplicated_spam_dataset (1).xlsx'  # Replace with your actual file path
df = pd.read_excel(file_path)

# Preprocess function
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and non-alphabetic words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Convert to lowercase
    return ' '.join(tokens)

# Apply preprocessing to 'v2' column (text column)
df['processed_text'] = df['v2'].apply(preprocess_text)

# Check the result
print(df[['v1', 'v2', 'processed_text']].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
import nltk

# Download the correct punkt resource
nltk.download('punkt')

# Now retry running your previous code
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download necessary NLTK resources
nltk.download('stopwords')

# Load your Excel file (adjust the path as needed)
file_path = '/content/updated_duplicated_spam_dataset (1).xlsx'  # Replace with your actual file path
df = pd.read_excel(file_path)

# Preprocess function
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and non-alphabetic words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Convert to lowercase
    return ' '.join(tokens)

# Apply preprocessing to 'v2' column (text column)
df['processed_text'] = df['v2'].apply(preprocess_text)

# Check the result
print(df[['v1', 'v2', 'processed_text']].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


    v1                                                 v2  \
0  ham          जोस पूछते हैं कि क्या आप मिलना चाहते हैं?   
1  ham  ఆయన అక్కడే ఉన్నారు. మీరు అతడికి ఫోన్ చేసి కలుస...   
2  ham  ఉపాధ్యాయుడి కంటే జీవితం చాలా కఠినమైనది ... Bco...   
3  ham  बहुत खूब। आप ठीक कह रहे हैं! मुझे ऐसा करने का ...   
4  ham               आप अपनी बाइक की सवारी कब कर रहे हैं?   

                          processed_text  
0                                     आप  
1                                    ఆయన  
2                           Bcoz amp amp  
3  आप कह मतलब एक जब और बदल बदल पर cuz यह  
4                               आप कब कर  


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Step 1: Vectorize the processed text
vectorizer = TfidfVectorizer(max_features=5000)  # Adjust the number of features as needed
X = vectorizer.fit_transform(df['processed_text'])

# Step 2: Prepare labels (target) from the 'v1' column
y = df['v1']  # Assuming 'v1' contains the labels (ham/spam)

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# Step 5: Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Step 6: Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))



Accuracy: 0.8629441624365483
Classification Report:
               precision    recall  f1-score   support

         ham       0.86      0.96      0.91       278
        spam       0.88      0.62      0.73       116

    accuracy                           0.86       394
   macro avg       0.87      0.79      0.82       394
weighted avg       0.86      0.86      0.86       394

